# Load data
This code loads the data for 1 subject with the help of methods defined in data_handler.ipynb.
Also prints distribution of the split just to make sure it's stratified.

In [2]:
import ipynb
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_data, split_info


data, description = load_subject(1)
labels = extract_labels(description)
train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data, labels)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[99, 99, 99, 99] 
 [13, 13, 14, 14] 
 [13, 13, 12, 12]
25.0 %  25.0 %  25.0 %  25.0 % 
24.074074074074073 %  24.074074074074073 %  25.925925925925924 %  25.925925925925924 % 
26.0 %  26.0 %  24.0 %  24.0 % 


# EEGNet
Implementation of EEGNetfrom (Lawhern et. al.) for the use on "Thinking out loud" dataset.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
import copy

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.f1 = 16
        self.d = 4
        self.f2 = self.d*self.f1

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = self.f1, kernel_size = (1, 128), padding = 0, bias = False) # 128 = half sampling rate, reduces to 2Hz, 16 is probably F1, the number of filters (output channels)
        self.batchnorm1 = nn.BatchNorm2d(self.f1, False)

        self.conv2 = nn.Conv2d(in_channels = self.f1, out_channels = self.f1*self.d, kernel_size = (128, 1), padding = 0, bias = False) # row, column] 128 = number of sensors for the depth wise convolution
        self.batchnorm2 = nn.BatchNorm2d(self.f1*self.d, False) # Not sure this is the right number of features.
        self.elu = nn.ELU()
        self.avgPool1 = nn.AvgPool2d((1,4)) # Reduces sampling rate to 64hz

        # Block 2

        # Separable convolution
        self.depthwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f1*self.d, kernel_size = (1, 32),
                        bias=False, groups = self.f1*self.d, padding = (0, 16//2)) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        self.pointwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f2, kernel_size = 1) # This results in f2 channels of the output here.
        
        self.batchnorm3 = nn.BatchNorm2d(self.f2, False)

        self.avgPool2 = nn.AvgPool2d((1,8))
        
        # Fully connected classifier
        self.classifier = nn.Linear(1920,4, bias = False)
        self.softmax = nn.Softmax()


    def forward(self, x, dropout = 0.5): # Dropout 0.5 dropout for within-subject and 0.25 for cross-subject
        # Block 1
        res = self.conv1(x)
        res = self.batchnorm1(res)
        res = self.conv2(res)
        res = torch.renorm(res, p=2, dim=0, maxnorm=1)
        res = self.batchnorm2(res)
        #res = F.dropout(0.25)
        res = self.elu(res)
        res = self.avgPool1(res)
        res = F.dropout(res, dropout)
        # Block 2
        res = self.depthwise(res)
        res = self.pointwise(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.avgPool2(res)
        res = F.dropout(res, dropout)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res

    def train_model(self, train_data, train_labels, val_data, val_labels, epochs, batch_size, loss_func, optimizer):
        print("Epoch\t train loss\t validation loss")
        best_model = copy.deepcopy(self.state_dict())
        best_loss = 10000 # bad value for dis, validation loss comparison

        for epoch in range(epochs):        
            epoch_train_loss = 0
            epoch_val_loss = 0
            self.train() # Set model to train mode
            
            for i in range(len(train_data)//batch_size): # BATCH SIZE MUST BE EVEN DIVIDER OF DATA LEN, otherwise we miss stuff here
                start = i*batch_size
                end = (i+1)*batch_size

                train_inputs = torch.FloatTensor(train_data[start:end])
                train_truth = torch.LongTensor(train_labels[start:end])
                train_outputs = self(train_inputs)

                loss = loss_func(train_outputs, train_truth)
                
                #print("LOSS: ", loss)
                epoch_train_loss += loss
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            

            self.eval() # Set model to evaluation mode

            val_inputs = torch.FloatTensor(val_data)
            val_truth = torch.LongTensor(val_labels)
            val_outputs = self(val_inputs)
            val_loss = loss_func(val_outputs, val_truth)
            epoch_val_loss += val_loss

            # Check for new best model, this should be on val_data instead
            if epoch_val_loss < best_loss:
                best_model = copy.deepcopy(self.state_dict())

            print(epoch, "\t ", epoch_train_loss.item(), "\t", epoch_val_loss.item())

        self.load_state_dict(best_model) # Set model to best performing one.
                



# TODO: Run on cuda.
# TODO: Add validation data and labels to train_model method params
# TODO: Implement early stopping.
# TODO: Graphing of train and val loss.

## Testing code
Code for testing the EEGNet implementation

In [3]:
train_data = torch.tensor(train_data)
train_data = torch.unsqueeze(train_data,1).float()
train_labels = torch.tensor(train_labels).long()

val_data = torch.tensor(val_data)
val_data = torch.unsqueeze(val_data,1).float()
val_labels = torch.tensor(val_labels).long()

test_data = torch.tensor(test_data)
test_data = torch.unsqueeze(test_data,1).float()
test_labels = torch.tensor(test_labels).long()


# Uncomment this if running on only 1 trial
#train_data = torch.unsqueeze(torch.unsqueeze(d,0),0).float() # Unsqueeze adds a wrapper dimension of 1s


In [4]:
# Dumb checking of initial weights
def accuracy_check(data, labels):
    r = network.forward(data)
    p = torch.max(r,1)[1]
    correct = 0.0
    for i in range(len(p)):
        if p[i] == labels[i]:
            correct += 1
    print("ACCURACY:", correct/len(p))


network = EEGNet().float()
accuracy_check(train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
network.train_model(train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 5, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(train_data, train_labels)

/tmp/ipykernel_64742/4033856455.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)


ACCURACY: 0.17666666666666667
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  -7.370763301849365 	 -0.2506704032421112
1 	  -8.479423522949219 	 -0.2495395541191101
2 	  -9.203104972839355 	 -0.250101774930954
3 	  -9.738449096679688 	 -0.25820720195770264
4 	  -10.52703857421875 	 -0.24233919382095337
ACCURACY: 0.5666666666666667


In [5]:
accuracy_check(train_data, train_labels)
accuracy_check(val_data, val_labels)
accuracy_check(test_data, test_labels)

/tmp/ipykernel_64742/4033856455.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)


ACCURACY: 0.55
ACCURACY: 0.27
ACCURACY: 0.24


In [22]:
print(val_labels)

t = [0,0,0,0]
v = [0,0,0,0]
te = [0,0,0,0]

for l in train_labels:
    t[l] +=1

for l in val_labels:
    v[l] +=1

for l in test_labels:
    te[l] +=1

print(t,"\n", v, "\n", te)

print(t[0]/len(train_data)*100, "% ", t[1]/len(train_data)*100, "% ",t[2]/len(train_data)*100, "% ",t[3]/len(train_data)*100, "% ")
print(v[0]/len(val_data)*100, "% ", v[1]/len(val_data)*100, "% ",v[2]/len(val_data)*100, "% ",v[3]/len(val_data)*100, "% ")
print(te[0]/len(test_data)*100, "% ", te[1]/len(test_data)*100, "% ",te[2]/len(test_data)*100, "% ",te[3]/len(test_data)*100, "% ")

[1 3 3 1 1 1 1 1 2 2 3 2 2 1 2 0 0 1 0 0 3 1 2 3 3 2 2 2 2 0 2 2 3 0 3 2 0
 0 0 1 1 0 3 3 3 0 1 0 0 3 3 1 2 3]
[99, 99, 99, 99] 
 [13, 13, 14, 14] 
 [13, 13, 12, 12]
25.0 %  25.0 %  25.0 %  25.0 % 
24.074074074074073 %  24.074074074074073 %  25.925925925925924 %  25.925925925925924 % 
26.0 %  26.0 %  24.0 %  24.0 % 
